In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Nuclear energy related deaths

In [10]:
deaths_df = pd.read_csv('nuclear_data/rates_death_from_energy_production_per_twh.csv')
deaths_df = deaths_df.sort_values(by='Deaths per TWh of electricity production', ascending=False)
deaths_df.head()

,Entity,Deaths per TWh of electricity production,Year
1,Brown coal,32.720,2021
2,Coal,24.620,2021
6,Oil,18.430,2021
0,Biomass,4.630,2021
3,Gas,2.821,2021


In [11]:
fig = px.bar(
    deaths_df,
    x='Entity',
    y='Deaths per TWh of electricity production',
    color='Deaths per TWh of electricity production',
    color_continuous_scale='RdYlGn_r'  # correcte naam
)

fig.update_layout(
    title="Death-rate per TWh produced per energy source",
    xaxis_title="Energy production method",
    yaxis_title="Deaths per TWh",
    coloraxis_showscale=False,
)

fig.show()

## Nuclear Waste

An important aspect of nuclear energy is nuclear waste. Nuclear reactors produce highly radioactive waste that needs to be stored for decades on end

In [12]:
# data from https://www.eia.gov/nuclear/spent_fuel/ussnftab3.php
waste_df = pd.read_csv('nuclear_data/nuclear_spent_fuel.csv')

In [13]:
# calulate average burnup per year
waste_df['total_average_burnup'] = (
    (
        (waste_df['bwr_average_burnup'] * waste_df['bwr_tons_uranium']) +
        (waste_df['pwr_average_burnup'] * waste_df['pwr_tons_uranium'])
    ) /
    (waste_df['bwr_tons_uranium'] + waste_df['pwr_tons_uranium'])
)

In [14]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=waste_df['year'],
        y=waste_df['bwr_average_burnup'],
        name='Boiling water reactor',
        mode='lines+markers',
        line=dict(
            color="#7fc97f"
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=waste_df['year'],
        y=waste_df['pwr_average_burnup'],
        name='Presurised water reactor',
        mode='lines+markers',
        line=dict(
            color="#fdc086"
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=waste_df['year'],
        y=waste_df['total_average_burnup'],
        name='Total',
        mode='lines+markers',
        line=dict(
            color="#beaed4"
        ),
    )
)

fig.update_layout(
    title="Average Burnup per reactor method",
    xaxis_title="Year",
    yaxis_title="Burnup (GWd/MTU)",
)

fig.show()

In [15]:
fig = go.Figure(
    go.Scatter(
        x=waste_df['year'],
        y=waste_df['total_tons_uranium'],
        mode='lines+markers',
        line=dict(
            color="#298f0d"
        ),
    )
)

fig.update_layout(
    title="Total Tons of Uranium Used",
    xaxis_title="Year",
    yaxis_title="MTU",
)
fig.show()